Prototyping notebook for predicting stock volaitility, prices, etc using extra data from web trends, news, etc. 

In [22]:
# %pip install compress_pickle[lz4] pandas numpy matplotlib yahooquery

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from compress_pickle import dump, load
import os
from yahooquery import Ticker
import timeit
import time
import datetime
from aquire_stock_search_terms import aquire_stock_search_terms as aquire_terms

1) Download and extract https://www.kaggle.com/datasets/footballjoe789/us-stock-dataset

2) Copy Stocks/ to data/. You can delete the other files as they are not used (yet, at least).

In [24]:
search_terms = aquire_terms('data/Stocks/')


In [25]:
search_terms.data

,Ticker,Company,Executive,Industry,quoteType,sector
0,A,"Agilent Technologies, Inc.",Michael McMullen,Diagnostics & Research,EQUITY,Healthcare
1,AA,Alcoa Corporation,Roy Harvey,Aluminum,EQUITY,Basic Materials
2,AAC,Ares Acquisition Corporation,David Kaplan,Shell Companies,EQUITY,Financial Services
3,AACG,ATA Creativity Global,Xiaofeng Ma,Education & Training Services,EQUITY,Consumer Defensive
4,AACI,Armada Acquisition Corp. I,Stephen Herbert,Shell Companies,EQUITY,Financial Services
...,...,...,...,...,...,...
7677,ZVSAW,None,,,EQUITY,
7678,ZWS,Zurn Elkay Water Solutions Corporation,Todd Adams,Pollution & Treatment Controls,EQUITY,Industrials
7679,ZYME,Zymeworks Inc.,Ali Tehrani PhD,Biotechnology,EQUITY,Healthcare
7680,ZYNE,"Zynerba Pharmaceuticals, Inc.",Armando Anido MBA,Drug Manufacturers—Specialty & Generic,EQUITY,Healthcare


In [26]:
count = 0
for ticker in range(len(search_terms.data)):
    if search_terms.data['Ticker'][ticker] != "TSLA":
        count += 1
    else:
        print(search_terms.data['Company'][ticker])
        print(search_terms.data['Ticker'][ticker])
        print(ticker)
        break

Tesla, Inc.
TSLA
6981


In [27]:
a = search_terms.data.iloc[6:7].Ticker.values[0]
ttt = Ticker(a)

In [28]:
ttt.quote_type[a]
{**ttt.asset_profile[a], **ttt.quote_type[a]}
#self.yh_tickers[t] = {**self.yh_tickers[t], **more_info}

{'address1': '6862 Elm Street',
 'address2': 'Suite 320',
 'city': 'McLean',
 'state': 'VA',
 'zip': '22101-1720',
 'country': 'United States',
 'phone': '703 373 0200',
 'fax': '703 373 0680',
 'website': 'https://www.arlingtonasset.com',
 'industry': 'REIT—Mortgage',
 'sector': 'Real Estate',
 'longBusinessSummary': 'Arlington Asset Investment Corp. (NYSE: AAIC) currently invests primarily in mortgage related and residential real estate and has elected to be taxed as a REIT. The Company is headquartered in the Washington, D.C. metropolitan area. For more information, please visit www.arlingtonasset.com.',
 'fullTimeEmployees': 11,
 'companyOfficers': [{'maxAge': 1,
   'name': 'Mr. J. Rock Tonkel Jr., CPA',
   'age': 59,
   'title': 'Pres, CEO & Director',
   'yearBorn': 1963,
   'fiscalYear': 2021,
   'totalPay': 1618545,
   'exercisedValue': 0,
   'unexercisedValue': 0},
  {'maxAge': 1,
   'name': 'Mr. Richard E. Konzmann',
   'age': 54,
   'title': 'Exec. VP, Treasurer & CFO',
   '

In [29]:
officers = search_terms.yh_tickers[6436]
officers = search_terms.yh_tickers[7]
officers

{'exchange': 'NYQ',
 'quoteType': 'EQUITY',
 'symbol': 'AAIN',
 'underlyingSymbol': 'AAIN',
 'shortName': 'Arlington Asset Investment Corp',
 'longName': None,
 'firstTradeDateEpochUtc': '2021-07-19 08:30:00',
 'timeZoneFullName': 'America/New_York',
 'timeZoneShortName': 'EST',
 'uuid': '6a83b0b9-f174-3be1-940d-58834cd7fd8d',
 'gmtOffSetMilliseconds': -18000000,
 'maxAge': 1}

In [30]:
# For every stock ticker symbol in stock_symbols, lookup the top executive and the company name.


Datasets:
We need a historical dataset of stock prices.
Secondly, we need a dataset of web trends for the same time period. 

On the other hand, once the model is trained, it needs to be able to aquire live data that looks the same as the training data.

Possible sources:
   - twitter
   - google trends

Possible models:
    - Train a model to process text into an encoding
    - Train a model to do stock price prediction
    - Combine the two models to predict stock price/volatility (not sure which is best) from text with greater accuracy